# Table of Contents<a id="Top"></a>

1. [Problem Statement](#1)<br>
2. [Sensitivity Analysis](#5)<br>
    2.1 [TUC - BOS](#5.2)<br>

# 1. Problem Statement<a id=1></a>

In this problem, we will consider a sensitivity analysis related to Tucson to Boston. Specifically, let's examine how changes in the cost per unit coefficient changes certain aspects of this optimization problem. This will be similar to the work that we did in class related to the Tucson to Atlanta route.

If you have the figure `problem.png` in the same folder as this .ipynb, you can see the picture that uses this code: `![Table of Data](problem.png)`

![Table of Data](problem.png)

##### [Back to Top](#Top)

# 2. Data<a id=2></a>

In [ ]:
import pandas as pd 
import pyomo.environ as pe
import matplotlib.pyplot as plt
import seaborn as sns

### Read and convert data

In [ ]:
xlsx_file = pd.ExcelFile('w05-c02-transportation.xlsx')
xlsx_file.sheet_names

Show the tab and shift-tab tricks. 

In [ ]:
cost = pd.read_excel(xlsx_file, sheet_name = 'coef', index_col = 0)
cost

In [ ]:
demand = pd.read_excel(xlsx_file, sheet_name = 'rhs', index_col = 0, usecols = [0,1])
capacity = pd.read_excel(xlsx_file, sheet_name = 'rhs', index_col = 0, 
                         usecols = [3,4], nrows = 3)

In [ ]:
demand

In [ ]:
capacity

##### [Back to Top](#Top)

# 3. Model Definition<a id=3><a>

In [ ]:
model = pe.ConcreteModel()

### Define Decision Variables

We will create three plant variables with individual indexes for the four warehouses. 

In [ ]:
DV_indexes = ['ATL', 'BOS', 'CHI', 'DEN']
model.MIN = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.PIT = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.TUC = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.pprint()

### Define Objective Function

Here we need to create a formula for all 12 decision variables. We loop through the warehouse indexes for each plant.

In [ ]:
model.obj = pe.Objective(expr = sum([cost.loc['MIN', index]*model.MIN[index] for index in DV_indexes] +
                         [cost.loc['PIT', index]*model.PIT[index] for index in DV_indexes] +
                         [cost.loc['TUC', index]*model.TUC[index] for index in DV_indexes]),
                         sense = pe.minimize)

In [ ]:
model.obj.pprint()

### Define Constraints

We finish defining the model by defining both the capacity and demand constraints.

In [ ]:
#Capacity Constraints
model.con_MIN = pe.Constraint(expr = sum(model.MIN[index] for index in DV_indexes) 
                              <= capacity.loc['MIN','Capacity'])
model.con_PIT = pe.Constraint(expr = sum(model.PIT[index] for index in DV_indexes)
                              <= capacity.loc['PIT','Capacity'])
model.con_TUC = pe.Constraint(expr = sum(model.TUC[index] for index in DV_indexes)
                              <= capacity.loc['TUC','Capacity'])   
#Demand Constraints
model.con_ATL = pe.Constraint(expr = model.MIN['ATL'] + model.PIT['ATL']+ 
                              model.TUC['ATL'] >= demand.loc['ATL','Requirement'])
model.con_BOS = pe.Constraint(expr = model.MIN['BOS'] + model.PIT['BOS']+ 
                              model.TUC['BOS'] >= demand.loc['BOS','Requirement'])
model.con_CHI = pe.Constraint(expr = model.MIN['CHI'] + model.PIT['CHI']+ 
                              model.TUC['CHI'] >= demand.loc['CHI','Requirement'])
model.con_DEN = pe.Constraint(expr = model.MIN['DEN'] + model.PIT['DEN']+ 
                              model.TUC['DEN'] >= demand.loc['DEN','Requirement'])

In [ ]:
model.con_MIN.pprint()

##### [Back to Top](#Top)

# 4. Model Solution<a id=4></a>

In [ ]:
opt = pe.SolverFactory('glpk')
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)

And here we show the final values for the model shown in the constraints as `Body`.
Note we can see the final values for our demand and capacity constraints. All of our lhs values are at the bounds so are binding constraints except for capacity constraint 3 which had a final value of 14,000 but the capacity is 15,000 so it had a slack of 1,000 units.

In [ ]:
model.display()

The above shows us all the information all at once. Let's pull out the optimal minimum cost and the final value of the decision variables.

#### Optimal Objective Value

In [ ]:
obj_val = model.obj.expr()
print(f'optimal objective value minimum cost = ${obj_val:.2f}')

#### Optimal Decision Variables

In order to capture the results, we have to use new code to reference the 3 variable names - `model.component_objects(pe.Var)` and each set of indexes. We can use looping to pull out the values. 

In [ ]:
for DV in model.component_objects(pe.Var):
    print(DV)
    for var in DV:
        print(" ", var, DV[var].value)

Let's do create a `pd.DataFrame` to store the results of our solution. 

In [ ]:
DV_solution = pd.DataFrame()
for DV in model.component_objects(pe.Var):
    for var in DV:
        DV_solution.loc[DV.name, var] = DV[var].value
DV_solution

Here we create a plot of the Decision Variables solution.

# 5. Sensitivity Analysis<a id=5></a>

Define a new function `run_model()` to obtain values.

In [ ]:
def run_model():
    #Once we solve the model we return the model object so we can get the optimal obj function value
    return model

## 5.1 TUC-BOS<a id=5.1></a>

Here is the original cost table just to remind us.

In [ ]:
cost

First we'll loop through all the costs from shipping from Tucson to Boston, re-solve the model, and capture the objective function optimal values. Create a sequence from 0.5 to 0.7 in increments of 0.1.

In [ ]:
tuc_bos_costs = []
tuc_bos_costs

In [ ]:
obj_list_tb = []
for val in tuc_bos_costs:
    #This single line updates the cost table
    #This reruns the model using our function
obj_list_tb

Let's make it look nicer and easier to see what is going on by storing the results in a dataframe changing the index to be the costs.

In [ ]:
obj_df_tb = pd.DataFrame(obj_list_tb, 
                         index = tuc_bos_costs, 
                         columns = ['cost'])
obj_df_tb

Plot the results using Seaborn

In [ ]:
zip_list = 
zip_list

In [ ]:
df = pd.DataFrame(zip_list, columns = ['TUC2BOS', 'Cost'])
df

In [ ]:
plt.figure(figsize=(8,5))
plt.show()

# Create the line plots

So what about the Decision variables? There are 12 for each model solution, we need to capture them a slightly different way. We'll put each solution in a list and then make a list of lists.

In [ ]:
DV_list_tb=[]
for val in tuc_bos_costs:
    
print(DV_list_tb)

Again, let's format this so it is easy to read.

In [ ]:
DV_col_names=['M,A','M,B','M,C','M,D','P,A','P,B','P,C','P,D', 'T,A','T,B','T,C','T,D']
DV_df_tb

We'll finish looking at this data by creating a line plot to monitor the changes in the optimal DV over the different Tucson-Boston cost changes. So - this kinda works but we have too
many decision variables it is hard to follow the colors.

In [ ]:
plt.figure(figsize=(8,5))
lt.show()

For our final version, let's create a plot for each Plant's Decision Variables. Remember we stored the Plant,Warehouse pairs for the Decision Variables in the list `DV_col_names`. Create the line plots showing the change in objective function for changes in the shipment costs associated with the Tucson to Boston route that we defined above. Make a plot showing all lines on the same figure and another showing the change in costs facetted.  

In [ ]:
df = pd.melt(DV_df_tb, 'Costs')
df

In [ ]:
plt.figure(figsize=(8,5))
plt.show()

In [ ]:
df['plants'] = df['variable'].str[:1]
df

In [ ]:
sns.set_style({'axes.grid' : True,'axes.edgecolor':'black'})
plt.show()